In [ ]:
import pandas as pd
import numpy as np

def feature_engineering_test(df):
    """
    Applica le stesse trasformazioni del Train al Test set.
    Ricrea le colonne mancanti (aggregazioni, incroci, ratio).
    """
    # Lavoriamo su una copia per non modificare l'originale
    df = df.copy()

    # =================================================================
    # 1. RICREAZIONE GRUPPI DI SINTOMI (Deduotti dalle colonne test)
    # =================================================================

    # DEFINISCI QUI LE LISTE DI COLONNE CHE FORMANO I GRUPPI
    # (Ho inserito quelle logiche basandomi sulla tua lista 'test')

    # Gruppo Respiratorio
    cols_respiratory = [
        'curill_respiratory', 'curill_pneumonia', 'curill_bronchitis',
        'curill_sinus', 'curill_flu', 'history_asthma', 'history_copd'
    ]
    # Filtriamo solo quelle che esistono davvero nel df attuale
    cols_respiratory = [c for c in cols_respiratory if c in df.columns]

    # Gruppo Cardiaco (Ho messo le storie cliniche cardiache presenti)
    cols_cardiac = [
        'history_cardiac', 'history_heart', 'history_coronary',
        'history_hypertension', 'meds_antihypertensive'
    ]
    cols_cardiac = [c for c in cols_cardiac if c in df.columns]

    # Gruppo Neurologico
    cols_neuro = [
        'history_neurologic', 'history_stroke', 'history_seizure'
    ]
    cols_neuro = [c for c in cols_neuro if c in df.columns]

    # Gruppo Febbre
    cols_fever = ['curill_fever']
    cols_fever = [c for c in cols_fever if c in df.columns]

    # --- CALCOLO I NUMERI (Counts) ---
    df['num_symp_respiratory'] = df[cols_respiratory].sum(axis=1)
    df['num_symp_cardiac']     = df[cols_cardiac].sum(axis=1)
    df['num_symp_neurologic']  = df[cols_neuro].sum(axis=1)
    df['num_symp_fever']       = df[cols_fever].sum(axis=1)

    # --- CALCOLO I FLAG (Booleani: 1 se ha almeno un sintomo, 0 altrimenti) ---
    df['symp_respiratory'] = (df['num_symp_respiratory'] > 0).astype(int)
    df['symp_cardiac']     = (df['num_symp_cardiac'] > 0).astype(int)
    df['symp_neurologic']  = (df['num_symp_neurologic'] > 0).astype(int)
    df['symp_fever']       = (df['num_symp_fever'] > 0).astype(int)

    # --- CALCOLO TOTALE SINTOMI CALCOLATI ---
    # Nota: Usiamo questo come denominatore per i ratio
    df['num_symp_total'] = (
        df['num_symp_respiratory'] +
        df['num_symp_cardiac'] +
        df['num_symp_neurologic'] +
        df['num_symp_fever']
    )

    # =================================================================
    # 2. FEATURE CROSSES (Moltiplicazioni)
    # =================================================================

    # Età x Numero Sintomi (Uso NUMERO_SINTOMI originale se presente, o quello calcolato)
    col_sintomi_base = 'NUMERO_SINTOMI' if 'NUMERO_SINTOMI' in df.columns else 'num_symp_total'

    df['fc_age_x_num_symptoms'] = df['AGE_YRS'] * df[col_sintomi_base]

    # History x Numero Sintomi
    if 'has_history' in df.columns:
        df['fc_history_x_num_symptoms'] = df['has_history'] * df[col_sintomi_base]
    else:
        # Fallback se has_history non c'è (creiamo una somma di history)
        history_cols = [c for c in df.columns if 'history_' in c]
        df['fc_history_x_num_symptoms'] = df[history_cols].max(axis=1) * df[col_sintomi_base]

    # Age x History Cardiac
    # Attenzione: Usiamo 'history_cardiac' se c'è, altrimenti la somma calcolata sopra
    if 'history_cardiac' in df.columns:
        df['fc_age_x_history_cardiac'] = df['AGE_YRS'] * df['history_cardiac']
    else:
        df['fc_age_x_history_cardiac'] = df['AGE_YRS'] * df['symp_cardiac']

    # =================================================================
    # 3. RATIOS (Divisioni)
    # =================================================================

    # Ratio Respiratory
    # Gestiamo la divisione per zero con numpy
    df['ratio_symp_respiratory'] = np.where(
        df['num_symp_total'] > 0,
        df['num_symp_respiratory'] / df['num_symp_total'],
        0
    )

    return df

# ======================================================
# UTILIZZO DELLO SCRIPT
# ======================================================

# 1. Carica il dataset di Test (o Validation) originale
# val = pd.read_csv("val_step6.csv") o il tuo dataframe attuale

print("Colonne prima del fix:", val.shape[1])

# 2. Applica la funzione
val_engineered = feature_engineering_test(val)

print("Colonne dopo il fix:", val_engineered.shape[1])

# 3. Verifica che ci siano tutte le colonne necessarie
missing_cols = [c for c in X_train.columns if c not in val_engineered.columns]
if len(missing_cols) == 0:
    print("✅ Tutto perfetto! Il Test set ha le stesse colonne del Train.")
else:
    print("❌ Mancano ancora queste colonne:", missing_cols)

# 4. Ora puoi procedere con X_val e y_val
X_val = val_engineered[X_train.columns] # Riordina esattamente come il train
y_val = val_engineered['IS_SEVERE']